Kelompok 6:
- Fawwas Aliy 235150300111009
- Briliiant Akhmad Assiddiqqy 235150301111045
- Andan Riski Mustari 235150301111002

## ETL Pipeline

Membangun pipeline ETL sederhana dari JSON ke CSV. Prosesnya mencakup extract dan flatten data JSON, transform dengan konversi suhu ke Fahrenheit serta perhitungan comfort index, lalu load hasil ke file CSV. Soal ini menilai pemahaman alur pemrosesan data mentah.

### Preparation

Impor library yang dibutuhkan yaitu pandas untuk mengelola data dan json untuk membaca data json.

In [1]:
import pandas as pd
import json

### Function

Fungsi berupa Extract, Transform, dan Load.

In [2]:
def extract_json_data(file_path):
    # baca file sensor
    with open(file_path, "r") as f:
        data = json.load(f)
    
    # baca nested array
    rows = []
    for sensor in data:
        sensor_id = sensor["sensor_id"]
        truck_id = sensor["truck_id"]
        for reading in sensor["readings"]:
            rows.append({
                "sensor_id": sensor_id,
                "truck_id": truck_id,
                "timestamp": reading["timestamp"],
                "temp": reading["temp"],
                "humidity": reading["humidity"]
            })

    df = pd.DataFrame(rows)
    return df

def transform_data(df):
    # ubah celsius ke fahrenheit
    df["temp"] = df["temp"] * 9/5 + 32

    # kalkulasi comfort index
    df["comfort_index"] = (df["temp"] / df["humidity"]) * 100
    df = df.round({"temp": 2, "comfort_index": 2})
    
    return df

def load_to_csv(df, output_path):
    # simpan ke path output
    df.to_csv(output_path, index=False)
    print(f"Data berhasil disimpan ke {output_path}")

### Main

Ambil raw data pada datasets/sensors_2024-10-13.json dan extract data tersebut.

In [3]:
raw_data = extract_json_data('datasets/sensors_2024-10-13.json')
print(raw_data.head())

  sensor_id truck_id             timestamp  temp  humidity
0   SENS001   TRK001  2024-10-13T08:00:00Z  25.0      65.0
1   SENS001   TRK001  2024-10-13T08:15:00Z  28.1      67.0
2   SENS001   TRK001  2024-10-13T08:30:00Z  26.7      66.0
3   SENS002   TRK002  2024-10-13T08:00:00Z   NaN      70.0
4   SENS002   TRK002  2024-10-13T08:15:00Z  22.3       NaN


Setelah di extract, beberapa parameter di transform terlebih dahulu.

In [4]:
transformed_data = transform_data(raw_data)
print(transformed_data.head())

  sensor_id truck_id             timestamp   temp  humidity  comfort_index
0   SENS001   TRK001  2024-10-13T08:00:00Z  77.00      65.0         118.46
1   SENS001   TRK001  2024-10-13T08:15:00Z  82.58      67.0         123.25
2   SENS001   TRK001  2024-10-13T08:30:00Z  80.06      66.0         121.30
3   SENS002   TRK002  2024-10-13T08:00:00Z    NaN      70.0            NaN
4   SENS002   TRK002  2024-10-13T08:15:00Z  72.14       NaN            NaN


Setelah itu output disimpan ke csv.

In [5]:
load_to_csv(transformed_data, 'etl_oupt.csv')

Data berhasil disimpan ke etl_oupt.csv
